In [14]:
import wfdb
import math
from wfdb import processing
rmeans={'filename':'rmean'}
# rinvalid=[]
def mean(file_path,file_name):
    file_destination=file_path+file_name
    sig, fields = wfdb.rdsamp(file_destination , channels=[0] ,sampto=40000 )
    xqrs = processing.XQRS(sig=sig[:,0], fs=fields['fs'])
    xqrs.detect()
    # print(fields)
    # wfdb.plot_items(signal=sig, ann_samp=[xqrs.qrs_inds] )

    #sqrs.qrs_inds is array containing indexes of qrs ?
    
    rpeaksum=0
    for i in xqrs.qrs_inds:
        rpeaksum+=sig[i]
    try:
        rmean=(rpeaksum/len(xqrs.qrs_inds))
        # print("mean for ",file_name , rmean)
        rmeans[file_name]=rmean
        
    except ZeroDivisionError:
        print("no r peaks found for",file_name)
        rmeans[file_name]="invalid"








    #finding standard deviation
    # rstdsum=0
    # for i in xqrs.qrs_inds:
    #     rstdsum+=(sig[i]-rmean)**2
    # rstd=math.sqrt(rstdsum/len(xqrs.qrs_inds))
    # print("standard deviation is",rstd)
    

def main():
    file_path=r"D:\vf predict\cu-ventricular-tachyarrhythmia-database-1.0.0"
    
    for i in range(1,36):
        file_name="\cu"+f"{i:02d}"
        mean(file_path,file_name)
    
    print(rmeans)
        
main()


Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
mean for  \cu01 [2.13662252]
Learning initial signal parameters...
Failed to find 8 beats during learning.
Initializing using default parameters
Running QRS detection...
QRS detection complete.
no r peaks found for \cu02
Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
mean for  \cu03 [0.9885571]
Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
mean for  \cu04 [0.854625]
Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
mean for  \cu05 [0.90949416]
Learning initial signal parameters...
Found 8 beats during learning. In

Area??

In [8]:
number = 35
print(f"{number:02d}")

35
